In [ ]:
# find and import the profile closest to your target profile

import os
import mesa_reader as mr

def getClosestModel(modelNumber, workingDirectory):
    """Returns the MESA profile with the closest model number in a specified directory.
    
    Keyword arguments:
    modelNumber -- target/desired model number
    workingDirectory -- target directory
    """
    print('Looking for the profile closest to model #' + str(modelNumber) + ' in ' + str(workingDirectory))
    filenames = []

    for root, dirs, files in os.walk(workingDirectory):
        for file in files:
            if file.endswith("profiles.index"):
                filenames.append(os.path.join(root, file))

    profiles = []
    # import files
    for file in filenames:
        i = mr.MesaProfileIndex(file)
        profiles.append(i)

    # find the closest model number - ugly but functional
    closest = 0
    diff = 1e10
    profilePath = ''
    j = 0
    for index in profiles:
        values = index.model_numbers
        profileNums = index.profile_numbers
        k=0
        for i in values:
            if (abs(modelNumber - i) < diff):
                diff = abs(modelNumber - i)
                closest = i
                og = filenames[j]
                og = og[:-14] # put together the file name given the directory
                profilePath = og + 'profile' + str(profileNums[k]) + '.data'
            k+=1
        j+=1

#     print('Actual model number: ' + str(closest))
#     print('Difference between target and actual model: ' + str(diff))
    print('File path: ' + str(profilePath))
#     print('')

    # import target profile
    p = mr.MesaData(profilePath)
    return p

In [ ]:
# find the model number with the max radius

def getMaxRadiusProfile(workingDirectory):
    """Returns the profile with the biggest radius in a directory.
    
    Keyword arguments:
    workingDirectory -- target directory
    """
    
    import os
    import mesa_reader as mr
    from heapq import nlargest
    import numpy as np

    filenames = []

    for root, dirs, files in os.walk(workingDirectory):
        for file in files:
            if file.endswith("history.data"):
                filenames.append(os.path.join(root, file))

    # for each file, go in and correct that quotation mark error
    # replace 10.14-2019 with 10.14-2019"
    for file in filenames:
        s = open(file).read()
        s = s.replace('10.14-2019 ', '10.14-2019"')
        f = open(file, 'w')
        f.write(s)
        f.close()

    # for each file, read it into a variable based on which log folder it's in
    hBRExists = False
    hARExists, hRExists = False, False
    htSBExists, htlgTExists = False, False
    hCExists, hFExists, hLExists = False, False, False

    for file in filenames:
        if 'before_remove' in file:
            hBR = mr.MesaData(file)
            hBRExists = True

        elif 'after_remove' in file:
            hAR = mr.MesaData(file)
            hARExists = True

        elif 'remove' in file:
            hR = mr.MesaData(file)
            hRExists = True

        elif 'to_si_burn' in file:
            htSB = mr.MesaData(file)
            htSBExists = True

        elif 'to_lgT_9.9' in file:
            htlgT = mr.MesaData(file)
            hlgTExists = True

        elif 'convert' in file:
            hC = mr.MesaData(file)
            hCExists = True

        elif 'finish' in file:
            hF = mr.MesaData(file)
            hFExists = True

        else:
            hL = mr.MesaData(file)
            hLExists = True

    # frankenstein the data together because MESA is MESA

    hModels = []
    hRadius = []

    if hBRExists:
        hRadius.append(hBR.log_R)
        hModels.append(hBR.model_number)

    if hRExists:
        hRadius.append(hR.log_R)
        hModels.append(hR.model_number)

    if hARExists:
        hRadius.append(hAR.log_R)
        hModels.append(hAR.model_number)

    if htSBExists:
        hRadius.append(htSB.log_R)
        hModels.append(htSB.model_number)

    if htlgTExists:
        hRadius.append(htlgT.log_R)
        hModels.append(htlgT.model_number)

    if hCExists:
        hRadius.append(hC.log_R)
        hModels.append(hC.model_number)

    # these are behaving weirdly
    # if hFExists:
    #     hRadius.append(hF.log_R)
    #     hModels.append(hF.model_number)

    if hLExists:
        hRadius.append(hL.log_R)
        hModels.append(hL.model_number)
    
    
    # find the model number for when the radius is at its maximum
    maxValues = []
    maxModelNumbers = []

    # find the biggest 5 in each history file
    for x in range(len(hRadius)):
        data = hRadius[x]
        maxValues.append(nlargest(1, data))
        index = nlargest(1, range(len(data)), key=lambda idx: data[idx])
        maxModelNumbers.append(hModels[x][index])

#     print(maxValues)
#     print(maxModelNumbers)
#     print('-----')
#     print(max(maxValues))
    modelNumber = int(maxModelNumbers[np.argmax(maxValues)])
#     print(modelNumber)

    # import desired profile

    # find all profile.index files
    filenames = []

    for root, dirs, files in os.walk(workingDirectory):
        for file in files:
            if file.endswith("profiles.index"):
                filenames.append(os.path.join(root, file))

    profiles = []
    # import files
    for file in filenames:
        i = mr.MesaProfileIndex(file)
        profiles.append(i)

    return getClosestModel(modelNumber, workingDirectory)